In [1]:
#1
import numpy as np
%pylab 
#style.use('ggplot')
#rcParams['figure.figsize'] = 12,8
#import numba as nb
from numba import jit
from numba import njit
from numba.core import types
from numba.typed import Dict

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import copy
import networkx as nx
from copy import copy, deepcopy

import pandas as pd
#import connectivity as ct

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


Matrix connectivity functions

In [2]:
#Erdos-Renyi connectivity matrix 
@jit(nopython = True)
def getRandomConnectivity(N, pct_connected):
    #Directed mapping
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < pct_connected):
                    M[i,j] = 1 #i connects to j
                else:
                    M[i,j] = 0
    return M

@jit(nopython = True)
def kill_reciprocal(N,M):
  for i in range(N):
    for j in range(i):
      if ((M[i,j] == 1) and (M[j,i] == 1)):
        q =np.random.randint(2)
        if (q==0):
          M[i,j] = 0
        else:
          M[j,i] = 0
  return(M)






#Addition of motives by completing the triangle
@jit(nopython=True)
def addmotives(N,M,l,nummotives):
    for m in range(nummotives):
        c = 0
        while (c == 0):
            a = 0
            while(a < 2):
                i = np.random.randint(N)
                a = int(l[i,0])
            j1 = 0
            j2 = 0
            while (j1 == j2):
                j1 = np.random.randint(a)+1
                j2 = np.random.randint(a)+1
            b = int(l[j1,0])
            d = 0
            for k in range(b):
                if (j2 == int(l[j1,k+1])):
                    d = 1
            if (d==0):
                l[j1,0] = l[j1,0] + 1
                l[j1,b + 1] = j2
                c=1
    return(l)
        


#Addition of motives by adding full triangle    
def addmotives2(N,M,nummotives):
    m = 0
    while(m<nummotives):
        i = np.random.randint(N)
        j = np.random.randint(N)
        k = np.random.randint(N)
        if (M[i,j]==0):
            M[i,j] = 1
            m = m + 1
        if (M[j,k]==0):
            M[j,k] = M[j,k] + 1
            m = m + 1
        if (M[k,i] == 0):
            M[k,i] = M[k,i]+1
            m = m + 1
    #l = matrixOfEdges(M,N)
    return(M)        

def numberofmotives(N,M,l):
    num = 0
    for i in range(N):
        a = int(l[i,0])
        for j in range(a):
            for k in range(a):
                l1 = int(l[i,j+1])
                l2 = int(l[i,k+1])
                if (M[l1,l2]==1):
                    num = num + 1
    return(num)





def findandkillmotives(N,M,l,numtokill):
    num = 0
    i = 0
    while ((i<N)and(num < numtokill)):
        a = int(l[i,0])
        for j in range(a):
            for k in range(a):
                l1 = int(l[i,j+1])
                l2 = int(l[i,k+1])
                if (M[l1,l2]==1):
                    num = num + 1
                    M[l1,l2] = 0
        i = i + 1
    return(M,num)
    
def getrandomvector(N,ics):
    V=zeros(N)
    p=ics*1.0/N
    for i in range(N):
        if (np.random.rand(1)<p):
            V[i]=1
    return(V)



@jit(nopython=True)
def matrixOfEdges(M,N):
    N = int(N)
    E=np.zeros(N**2).reshape(N,N)  #prepare array with zeros
    k=np.sum(M,0) #array with degree of each vertex
    for i in range(N):
        a=int(k[i]) #degree of the current vertex
        E[i,0]=a  #we put it to the zero row of matrix of edges
    for i in range(N):
        a=int(k[i])
        q=1
        for j in range(N):
            if (M[j,i]==1):
                E[i,q]=j #all the next elements in current column are number of vertices current vertex is connected to
                q=q+1
    return(E)


@jit(nopython=True)
def recovermatrixfromlist(E,N):
    q=0;
    M=np.zeros(N*N).reshape(N,N)
    for i in range(N):
        a=E[i,0]
        b=int(a)
        for j in range(b):
            k=E[i,j+1]
            l=int(k)
            M[i,l]=1
    return(M)


def doubleconstellation(N,k):
    M=np.zeros(N*N).reshape(N,N)
    for i in range(k):
        for j in range(k):
            M[i,j+k]=1            
    for i in range(N-2*k):
        for j in range(k):
            M[i+2*k,j]=1
            M[j+k,i+2*k]=1
    return(M)



def constellationwithrandom(N,k,p):
    Edges=(2*(N-2*k)*k-k*k)*1.0
    M=np.zeros(N*N).reshape(N,N)
    for i in range(k):
        for j in range(k):
            M[i,j+k]=1            
    for i in range(N-2*k):
        for j in range(k):
            M[i+2*k,j]=1
            M[j+k,i+2*k]=1
    p=p-Edges*1.0/(N*N-N)
    Q = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (Q[i,j] < p):
                    M[i,j] = 1 #i connects to j
    return(M)


def constantdegree(N,k):
    M=zeros(N*N).reshape(N,N)
    for i in range(N):
        for j in range(k):
            m=np.random.randint(N)
            M[i,m]=1
    return(M)


def coordinates(A,m,d):
    X=A
    coords=np.zeros(d)
    remainder=np.zeros(d)
    for i in range(d):
        (X,coords[i])=divmod(X,m)
    return(coords)

def distance(A,B,m,d):
    coordsa = coordinates(A,m,d)
    #print(coordsa)
    coordsb = coordinates(B,m,d)
    #print(coordsb)
    vecdist=np.zeros(d)
    for i in range(d):
        vecdist[i]=(coordsa[i]-coordsb[i])**2
    dist=np.sum(vecdist)
    return(dist)

def localmatr(m,d,p,a):
    N=m**d
    M = np.random.rand(N**2).reshape(N,N)
    for i in range(N):
        for j in range(N):
            if (i == j):
                M[i,j] = 0
            else:
                if (M[i,j] < p*np.exp(-a*distance(i,j,m,d))):
                    #print(distance(i,j,m,d))
                    M[i,j] = 1 #i connects to j
                else:
                    M[i,j] = 0
    return M

    


def ranvec(N,n):
    vec=np.zeros(N)
    for i in range(n):
        a=np.random.randint(N)
        while(vec[a]==1):
            a=np.random.randint(N)
        vec[a]=1
    return(vec)

def random_initial_state(N,n_times,n):
    states = np.zeros(N*n_times).reshape(N,n_times) - 1
    vec = ranvec(N,n)
    for i in range(N):
        states[i,0] = -1  - vec[i]
    return(states)

def add_one_active(N,vec):
    i = np.random.randint(N)
    while(vec[i] == 1):
        i = np.random.randint(N)
    vec[i] = 1
    return(vec)

def vector_to_matrix(N,n_times,vec):
    states = np.zeros(N*n_times).reshape(N,n_times) - 1
    for i in range(N):
        states[i,0] = -1 - vec[i]
    return(states)


def spike_raster_plot(N, n_timesteps, stepsize, states):
    neuralData = []
    for i in range(N):
        neuralData.append([])
        for j in range(n_timesteps):
            if (states[i,j] == 0):
                neuralData[i].append(j*stepsize)
    return(neuralData)


Dynamic functions, the simplest approach: background noise just contributes to the potential. We assume it to contribute equally to all neurons, we do not take here into account variations in the noise, we assume all the neurons to be identical.The growth of the EPSP is instanateous in this model.   

In [3]:
#Here we are trying another approach by taking into account synaptic delay.

#The function that updates neuron's state based on its current state and input.

#We are not modeling laser activated neurons as separate entities 
#but just as the same neuronsvwith additional input
@jit(nopython=True)
def neuron_state_update(state, inp, voltage, refr, threshold, EPSP_decay, tau_m):
    V = voltage * EPSP_decay + inp / tau_m
    if ((state < refr) or (V < threshold)):
        return(state + 1, V)
    else: 
        return(0,0)


@jit(nopython=True)
def neuron_state_update_noise(state, inp, voltage, refr, threshold, EPSP_decay, tau_m, noise_freq):
    V = voltage * EPSP_decay + inp / tau_m
    to_fire = np.random.rand()
    if (((state < refr) or (V < threshold)) and (to_fire > noise_freq) ):
        return(state + 1, V)
    else: 
        return(0,0)
    
@jit(nopython=True)    
def neuron_output(state,n_neighbors, W_synapse, tau_s, mu,  sigma):
    for i in range(n_neighbors):
        if (state == 0):
            W_synapse[i] = np.random.lognormal(mu, sigma)
            print(W_synapse[i])
    return(W_synapse*state*np.exp(-state / tau_s), W_synapse)
    
    

#It is not real excitation, but effective one
@jit(nopython=True) 
def excitation_by_laser(V, mu, sigma, n_spikes, n_timesteps):
    output = np.zeros(n_timesteps)
    j = 0
    for i in range(n_spikes):
        per = int(np.random.lognormal(mu, sigma))
        j = j + per
        output[j] = V
    return(output)




@jit(nopython = True)
def one_step_LIF(state, input_matrix, voltage,syn_del, W_syn,  edges, t, parameters):
    EPSP_decay = parameters['EPSP_decay'] # we multiply voltage to this constant every step to get exponential decay
    refr = int(parameters['refr']) 
    mu = parameters['mu'] 
    sigma = parameters['sigma']
    tau_s = int(parameters['tau_s'])
    tau_m = int(parameters['tau_m'])
    threshold = parameters['threshold'] #threshold for the transition
    n_neurons = int(parameters['n_neurons']) #total number of neurons
    
    for i in range(n_neurons):
        statei = state[i,t]
        n_neighbors = int(edges[i,0])
        W_synapse = np.zeros(n_neighbors)
        out = np.zeros(n_neighbors)
        constant = statei*np.exp(-statei / tau_s)
        for j in range(n_neighbors):
            out[j] = W_syn[i,j]*constant
        #out, W_synapse = neuron_output(statei,n_neighbors, W_synapse, tau_s, mu,  sigma)
            neighbor = int(edges[i, j+1])
            delay = int(syn_del[i,j])
            input_matrix[neighbor, t + delay + 1] = input_matrix[neighbor, t + delay + 1] + out[j]
    for i in range(n_neurons):
        state[i, t + 1], voltage[i, t + 1] = neuron_state_update(state[i, t ], input_matrix[i, t], voltage[i, t ], refr, threshold, EPSP_decay,tau_m)
        
    return(state, input_matrix, voltage)

@jit(nopython = True)
def create_noise_vector(N, fraction):
  nv = np.zeros(N)
  for i in range(N):
    p = np.random.rand()
    if (p < fraction):
      nv[i] = 1
  return(nv)

@jit(nopython = True)
def one_step_LIF_noise(state, input_matrix, voltage,syn_del, W_syn,  edges, t, parameters, noise_freq, noise_vector):
    #print(3.5)
    EPSP_decay = parameters['EPSP_decay'] # we multiply voltage to this constant every step to get exponential decay
    refr = int(parameters['refr']) 
    mu = parameters['mu'] 
    sigma = parameters['sigma']
    tau_s = int(parameters['tau_s'])
    tau_m = int(parameters['tau_m'])
    threshold = parameters['V_threshold'] #threshold for the transition
    n_neurons = int(parameters['n_neurons']) #total number of neurons
    #print(3.7)
    for i in range(n_neurons):
     #   print(3.75)
        statei = state[i,t]
        n_neighbors = int(edges[i,0])
        W_synapse = np.zeros(n_neighbors)
        out = np.zeros(n_neighbors)
        constant = statei*np.exp(-statei / tau_s)
        for j in range(n_neighbors):
            out[j] = W_syn[i,j]*constant
        #out, W_synapse = neuron_output(statei,n_neighbors, W_synapse, tau_s, mu,  sigma)
            neighbor = int(edges[i, j+1])
            delay = int(syn_del[i,j])
            input_matrix[neighbor, t + delay + 1] = input_matrix[neighbor, t + delay + 1] + out[j]
    for i in range(n_neurons):
      if (noise_vector[i] == 1):
        state[i, t + 1], voltage[i, t + 1] = neuron_state_update_noise(state[i, t ], input_matrix[i, t], voltage[i, t ], refr, threshold, EPSP_decay, tau_m, noise_freq)
      else:
        state[i, t + 1], voltage[i, t + 1] = neuron_state_update(state[i, t ], input_matrix[i, t], voltage[i, t ], refr, threshold, EPSP_decay, tau_m)
  

    return(state, input_matrix, voltage)

@jit(nopython=True)
def prepare_syndel(average, std, stepsize, edges, n_neurons ):
    mu, sigma = prepare_lognormal(average, std, stepsize)
    synaptic_del = np.zeros(n_neurons**2).reshape(n_neurons, n_neurons)
    for i in range(n_neurons):
        for j in range(n_neurons - 1):
            if (edges[i, j + 1] > 0):
                 syndel = np.random.lognormal(mu, sigma)
    return(synaptic_del)


@jit(nopython=True)
def prepare_syndel_2(average, std, stepsize, edges, n_neurons ):
    synaptic_del = np.zeros(n_neurons**2).reshape(n_neurons, n_neurons)
    for i in range(n_neurons):
      neigh = int(edges[i,0])
      for j in range(neigh):
        synaptic_del[i,j] = np.random.uniform(average - std, average + std)
    return(synaptic_del)


@jit(nopython = True)
def prepare_input(V, period, std, stepsize, n_neurons, n_timesteps, n_spikes, set_activated):
    input_matrix = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    n_activated = len(set_activated)
    mu, sigma = prepare_lognormal(period, std, stepsize)
    for i in range(n_activated):
        vec = excitation_by_laser(V, mu, sigma, n_spikes, n_timesteps)
        number = int(set_activated[i])
        for j in range(n_timesteps):
            input_matrix[number, j] = vec[j]
    return(input_matrix)
        
    

        
        

                
#The full process of synchronization, starting with particular set of activated neurons           
@jit(nopython=True)
def full_process( edges, syn_del, input_matrix, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector):
    #print(1)
    n_neurons = int(parameters['n_neurons'])
    stepsize = parameters['stepsize']
    
    tau_s = int(parameters['tau_s'])
    #print(2)
    states = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    for i in range(n_neurons):
        for j in range(n_timesteps):
            states[i,j] = 1000 * tau_s
    
                
    #        print(W_matrix[i,j])
    #excitation_by_laser(V, period, std, n_spikes, n_timesteps)
    max_del = np.max(syn_del.reshape(n_neurons**2))
    V_threshold = parameters['V_threshold'] 
    voltages = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    t = 0
    av_V = 0
    while ((t < n_timesteps - 1 - max_del) and (av_V < V_threshold)):
       # print(3)
        states, input_matrix, voltages = one_step_LIF_noise(states, input_matrix, voltages, syn_del, W_matrix,  edges, t, parameters, noise_freq, noise_vector)
      #  print(4)
        t = t + 1
        sum_V = 0
        for i in range(n_neurons):
            sum_V = sum_V + voltages[i,t]
        av_V = sum_V / n_neurons    
        #print(av_V)
    return(states, voltages, t)    



#The full process of synchronization, starting with particular set of activated neurons           
@jit(nopython=True)
def full_process_no_stop( edges, syn_del, input_matrix, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector):
    n_neurons = int(parameters['n_neurons'])
    stepsize = parameters['stepsize']
    
    tau_s = int(parameters['tau_s'])
    
    states = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    for i in range(n_neurons):
        for j in range(n_timesteps):
            states[i,j] = 1000 * tau_s
    
                
    #        print(W_matrix[i,j])
    #excitation_by_laser(V, period, std, n_spikes, n_timesteps)
    max_del = np.max(syn_del.reshape(n_neurons**2))
    voltages = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    t = 0
    av_V = 0
    while ((t < n_timesteps - 1 - max_del)):
        states, input_matrix, voltages = one_step_LIF_noise(states, input_matrix, voltages, syn_del, W_matrix,  edges, t, parameters, noise_freq, noise_vector)
        t = t + 1
        sum_V = 0
        for i in range(n_neurons):
            sum_V = sum_V + voltages[i,t]
        av_V = sum_V / n_neurons    
        #print(av_V)
    return(states, voltages, t)    

    



    

    
    
#@jit(nopython=True)
def averaging_over_trials(edges, syn_del, W_matrix,  parameters, V, period, stdlaser, n_activated, n_trials):
    n_neurons = int(parameters[8])
    stepsize = parameters[9]
    n_synch = 0
    t_sum = 0
    noise_freq = 0
    noise_fraction = 0.0 #fraction of noicy neurons

    noise_vector = create_noise_vector(N, noise_fraction)

    n_timesteps  = 10000
    max_del = np.max(syn_del.reshape(n_neurons**2))
    n_spikes = 7 
    #copy_states = states
    for i in range(n_trials):
        #if (copy_states == 0):
            #print('here', i)
            #states = random_initial_state(n_neurons, n_timesteps, n_activated)
        set_activated = active_set(N, n_activated)
        input_matrix = prepare_input2(V, period, stdlaser, stepsize, n_neurons, n_timesteps, n_spikes, set_activated)
        #states_old = np.copy(states)    
        states_new, voltages, t =  full_process( edges, syn_del, input_matrix, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector)
        print(t)
        #states = np.copy(states_old)
        if (t < n_timesteps - 2 - max_del):
            n_synch = n_synch + 1
            t_sum = t_sum + t
    if (n_synch == 0):
        return (0,0)
    else:
        return (n_synch*1.0/n_trials, t_sum * 1.0 / n_synch)
    
    
    
def averaging_over_trials_2(edges, syn_del, W_matrix,  parameters, V, period, stdlaser, set_activated, n_trials, noise_freq, noise_vector):
    n_neurons = int(parameters[8])
    stepsize = parameters[9]
    n_synch = 0
    t_sum = 0
    #noise_freq = 0
    #noise_fraction = 0.0 #fraction of noicy neurons

    noise_vector = create_noise_vector(N, noise_fraction)

    n_timesteps  = 10000
    max_del = np.max(syn_del.reshape(n_neurons**2))
    n_spikes = 7 
    #copy_states = states
    for i in range(n_trials):
        #if (copy_states == 0):
            #print('here', i)
            #states = random_initial_state(n_neurons, n_timesteps, n_activated)
        #set_activated = active_set(N, n_activated)
        input_matrix = prepare_input2(V, period, stdlaser, stepsize, n_neurons, n_timesteps, n_spikes, set_activated)
        #states_old = np.copy(states)    
        states_new, voltages, t =  full_process( edges, syn_del, input_matrix, W_matrix,  parameters, n_timesteps, noise_freq, noise_vector)
        #print(t)
        #states = np.copy(states_old)
        if (t < n_timesteps - 2 - max_del):
            n_synch = n_synch + 1
            t_sum = t_sum + t
    if (n_synch == 0):
        return (0,0)
    else:
        return (n_synch*1.0/n_trials, t_sum * 1.0 / n_synch)
    
    
#@jit(nopython=True)
def one_slice_experiment(edges, syn_del, W_matrix,  parameters, V, period, stdlaser, n_activated, n_trials):
    times = []
    probs = []
    n_neurons = int(parameters[8])
    pr = 1
    #print(type(std))
    while (pr > 0.15):
        pr, t = averaging_over_trials(edges, syn_del, W_matrix,  parameters, V, period, stdlaser, n_activated, n_trials)
        n_activated -= 1
        print(n_activated)
        print(pr)
        probs.append(pr)
        times.append(t)
    return(probs, times)


def one_slice_experiment_2(edges, syn_del, W_matrix,  parameters, V, period, stdlaser, set_activated, n_trials, noise_freq, noise_vector):
    times = []
    probs = []
    n_neurons = int(parameters[8])
    pr = 1
    #print(type(std))
    while (pr > 0.1):
        pr, t = averaging_over_trials_2(edges, syn_del, W_matrix,  parameters, V, period, stdlaser, set_activated, n_trials, noise_freq, noise_vector)
        #n_activated -= 1
        #print(n_activated)
        #print(pr)
        set_activated = remove_one_active(set_activated)
        probs.append(pr)
        times.append(t)
    return(probs, times)




def whospikes(N, n_timesteps, states):
    for i in range(n_timesteps):
        for j in range(N):
            if (states[j,i] == 0):
                print(i, "spikes", j)


        
@jit(nopython=True)           
def prepare_lognormal(av, std,stepsize):   
    sigmasq = np.log((std/av)**2 + 1) 
    sigma = np.sqrt(sigmasq)
    mu = np.log(av / stepsize)  - sigmasq / 2.0
    return(mu,sigma)    
    

@jit(nopython=True)                
def turn_data_into_scatter(N, n_tim, state):
    x = []
    y = []
    for j in range(n_tim):
        for i in range(N):
            if (state[i,j] == 0):
                x.append(i)
                y.append(j)
    return(x,y)
    
                
def probs_times(res, n_timesteps, n_trials, n_curves, n_max, stepsize):
    times = []
    probs = []
    for c in range(n_curves):
        t = res[c]
        times.append([])
        probs.append([])
        n_act = []
        for i in range(n_max):
            n_burst = 0
            sum_t = 0
            for j in range(n_trials):
                if (t[i,j] < n_timesteps - 1):
                    n_burst = n_burst + 1
                    sum_t = sum_t + t[i,j]                
            probs[c].append(n_burst * 1.0 / n_trials)
            if (n_burst > 0):
                sum_t = sum_t / n_burst
                times[c].append(sum_t * stepsize)
    return(times, probs)                     
            
def plotter(times, probs, n_curves, n_max):
    n_act = []
    n_shifted = np.zeros(n_max)
    for i in range (n_max):
        n_shifted[i] = i + 1
    for i in range(n_curves):
        
        q = len(times[i])
        n_act_vec = np.zeros(q)
        for j in range(q):
            n_act_vec[j] = n_max - q + j + 1
        n_act.append(n_act_vec)
    figure(1)
    for i in range(n_curves):
        plot(n_shifted, probs[i])
    figure(2)
    for i in range(n_curves):
        plot(n_act[i],times[i])
    
    
    
    
def list_from_matrices(N, edges, syn, W):
  e_l = []
  s_l = []
  W_l = []
  for i in range(N):
    neigh = int(edges[i,0])
    e_l.append([])
    s_l.append([])
    W_l.append([])
    for j in range(neigh):
      e_l[i].append(edges[i,j+1])
      s_l[i].append(syn[i,j])
      W_l[i].append(W[i,j])
  return(e_l,s_l,W_l)


def weight_matrix(n_neurons, edges, mu,sigma): 
   W_matrix = np.zeros(n_neurons**2).reshape(n_neurons, n_neurons)
   for i in range(n_neurons):
     neigh = int(edges[i,0])
     for j in range(neigh):
      W_matrix[i,j] = np.random.lognormal(mu, sigma)
   return(W_matrix)

def active_set(N, n_activated):
  set_activated = np.zeros(n_activated)
  for i in range(n_activated):
    a = np.random.randint(N)
    for j in range(i):
      while (a == set_activated[j]):
        a = np.random.randint(N)
    set_activated[i] = a
  return(set_activated)    

def remove_one_active(set_activated):
    l = len(set_activated)
    new_set = np.zeros(l-1)
    for i in range(l-1):
        new_set[i] = set_activated[i]
    return(new_set)
    
    

In [14]:
@njit
def my_params( ):
    stepsize = 0.05 #the size of the time step, in milliseconds
    refr = 3 / stepsize # refractory period of the neuron. Number before division is in milliseconds 
    tau_s = 0.5 / stepsize #tau_s from the equation for the current I = W * t * exp(-t/ tau_s). Number in ms.
    tau_m = 25 / stepsize #tau_m membrane time constant in milliseconds
    V_threshold = 12.0  #the threshold voltage in millivolts
    EPSP_decay = np.exp(-1.0 / tau_m )  # the factor by which the voltage decays following the equations
    V_EPSP = 2.8 #average magnitude of EPSP in mV
    std_EPSP = 0.01 #standard deviation of EPSP in mV
    W_average = 300 #approximate value of the parameter W in the equation for current (see above) in mV
    sigmasq = np.log((std_EPSP/V_EPSP)**2 + 1) #square of parameter sigma in lognormal distribution 
    sigma = np.sqrt(sigmasq) #parameter sigma in lognormal distribution
    mu = np.log(W_average) - sigmasq / 2.0  #parameter mu in lognormal distribution
    mu = mu + np.log(stepsize) #changing the stepsize goes into mu in this way
    noise_freq = 0.0005 * stepsize # frequency over the noise in kHz (1/ms)
    avdel = 1.3 / stepsize  #average synaptic delay time. In ms before division
    stddel = 1.1 / stepsize #standard deviation of the synaptic delay time. In ms before division
    n_neurons = 1000
    n_timesteps = 20000 #the 

    parameters = dict()
    parameters['EPSP_decay'] = EPSP_decay
    parameters['refr'] = refr
    parameters['mu'] = mu
    parameters['n_timesteps'] = n_timesteps
    parameters['sigma'] = sigma 
    parameters['tau_s'] = tau_s
    parameters['tau_m'] = tau_m
    parameters['V_threshold'] = V_threshold
    parameters['n_neurons'] = n_neurons
    parameters['stepsize'] = stepsize
    parameters['ER connection probability'] = 0.065
    parameters['Average synaptic delay'] = 1.3 / stepsize
    parameters['Standard deviation for synaptic delay'] = 1.1 / stepsize
    parameters['Laser strength'] = 20 * tau_m #magnitude of the input. Should be enough to make neuron to fire 
    parameters['Laser mean period'] = 39.2 / stepsize #average period of spiking after laser stimulation in ms, taken from frequency in Kaiwen paper
    parameters['Standard deviation of laser period'] = 4.7 / stepsize #standard deviation  of the period
    parameters['Number of spikes'] = 7 #number of spikes
    parameters['Number of laser activated neurons'] = 18  #number of activated neurons
    parameters['Mean delay for the first spike from laser activation'] = 20 / stepsize
    parameters['Standard deviation for the delay of the first spike'] = 3 / stepsize
    return parameters

# Parameters of the simulation

In [15]:
par = my_params()
N = int(par['n_neurons']) #total number of neurons

n_neurons = N

p = 0.065 #probability of the directed connection

mu = par['mu']

sigma = par['sigma']
stepsize = par['stepsize']
M = getRandomConnectivity(N,p) #prepare connectivity matrix of the network. In this case ER matrix.
edges = matrixOfEdges(M,N)  #other form of this matrix
W_matrix = weight_matrix(n_neurons, edges, mu,sigma) #matrix of synaptic weights

noise_freq = 0.0005 * stepsize # frequency over the noise in kHz (1/ms)
avdel = 1.3 / stepsize  #average synaptic delay time. In ms before division
stddel = 1.1 / stepsize
    
syn_del = prepare_syndel_2(avdel, stddel, stepsize, edges, n_neurons ) #matrix of synaptic delays

e_l, s_l , W_l = list_from_matrices(N, edges, syn_del, W_matrix) #write all the matrices in a form convenient for output



In [16]:
def generate_network(params, network_type = 'ER'):
    if (network_type=='ER'):
        N = int(params['n_neurons'])
        p = params['ER connection probability']
        mu = params['mu']

        sigma = params['sigma']
        M = getRandomConnectivity(N,p)
        edges = matrixOfEdges(M,N)
        W_matrix = weight_matrix(n_neurons, edges, mu,sigma) 
        avdel = params['Average synaptic delay']  
        stddel = params['Standard deviation for synaptic delay']
        syn_del = prepare_syndel_2(avdel, stddel, stepsize, edges, n_neurons ) #matrix of synaptic delays
    return(M, edges, W_matrix, syn_del)
    

    
@jit(nopython = True)
def prepare_input2(params, set_activated):
    n_neurons = int(params['n_neurons'])
    n_timesteps = int(params['n_timesteps'])
    print(1)
    input_matrix = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    n_activated = len(set_activated)
    for i in range(n_activated):
        vec = excitation_by_laser_2(params)
        number = int(set_activated[i])
        for j in range(n_timesteps):
            input_matrix[number, j] = vec[j]
    return(input_matrix)  

@jit(nopython=True) 
def excitation_by_laser_2(params):
    
    n_timesteps = int(params['n_timesteps'])
    output = np.zeros(n_timesteps)
    mdellas = params['Mean delay for the first spike from laser activation']
    stdellas = params['Standard deviation for the delay of the first spike']
    V = params['Laser strength']
    period = params['Laser mean period']
    deviation = params['Standard deviation of laser period']
    n_spikes = params['Number of spikes']
    j = int(np.random.normal(mdellas, stdellas))
    output[j] = V
    for i in range(n_spikes - 1):
        per = int(np.random.normal(period, deviation))
        j = j + per
        output[j] = V
    return(output)

In [26]:
M, edges, W_matrix, syn_del = generate_network(par)

# Parameters of the laser activation

In [27]:
N = int(par['n_neurons'])
n_activated = int(par['Number of laser activated neurons'])
set_activated = active_set(N, n_activated) #activated neurons are chosen
#set activated is a list of numbers. You also can give it explicitly, for example 
#set_activated = [1,2,3,4,5,6,7] 



# Creating the input matrix from laser

In [28]:
#line below - creates the input matrix (voltage vs time for each of the activated neurons)    
input_matrix = prepare_input2(par, set_activated)
saved_input = deepcopy(input_matrix)

1


#Printing activated neurons spiking times

In [29]:
@jit(nopython = True)
def compact_input(input_matrix, set_activated, par):
    n_timesteps = int(par['n_timesteps']) 
    n_spikes = int(par['Number of spikes']) 
    n_activated = len(set_activated)
    
    inp = np.zeros(n_activated * n_spikes).reshape(n_activated, n_spikes)
    for i in range(n_activated):
        number = int(set_activated[i])
        num_of_spike = 0
        for j in range(n_timesteps):
            
            #print(type(num_of_spike))
            if (input_matrix[number, j] > 0):
                k = int(num_of_spike)
                inp[i, k] = j
                num_of_spike =num_of_spike + 1
             #   print(type(num_of_spike))
    #print(inp)
    
    return(inp)




In [30]:
compact_input(input_matrix, set_activated, par)

array([[ 383., 1003., 1709., 2657., 3538., 4432., 5305.],
       [ 370., 1154., 2012., 2924., 3692., 4535., 5267.],
       [ 354., 1274., 1969., 2824., 3548., 4388., 5038.],
       [ 384., 1244., 2008., 2893., 3599., 4265., 4966.],
       [ 334., 1078., 1900., 2524., 3425., 4045., 4966.],
       [ 375., 1009., 1543., 2316., 3165., 3959., 4870.],
       [ 502., 1175., 1998., 2750., 3563., 4379., 5111.],
       [ 433., 1192., 2138., 2888., 3540., 4193., 5038.],
       [ 342., 1006., 1922., 2473., 3423., 4010., 4717.],
       [ 341., 1164., 1805., 2771., 3716., 4334., 5160.],
       [ 505., 1154., 2048., 3011., 3832., 4480., 5310.],
       [ 419., 1310., 2296., 2925., 3802., 4643., 5536.],
       [ 384., 1103., 2010., 2789., 3629., 4367., 5187.],
       [ 450., 1244., 2030., 2825., 3576., 4304., 5124.],
       [ 450., 1323., 2218., 2946., 3767., 4746., 5593.],
       [ 379., 1011., 1681., 2475., 3116., 3968., 4636.],
       [ 337., 1002., 1675., 2504., 3449., 4133., 4937.],
       [ 397.,

# Prepare the list with parameters

#Specifying which neurons can produce noise and which can not

In [31]:
noise_fraction = 0.0 #fraction of noicy neurons

noise_vector = create_noise_vector(N, noise_fraction)



In [ ]:
@jit(nopython=True)
def full_process(edges, syn_del, input_matrix, W_matrix,  par, noise_freq, noise_vector):
    n_neurons = int(par['n_neurons'])
    stepsize = par['stepsize']
    n_timesteps = int(par['n_timesteps'])
    tau_s = int(par['tau_s'])
    states = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    for i in range(n_neurons):
        for j in range(n_timesteps):
            states[i,j] = 1000 * tau_s
    max_del = np.max(syn_del.reshape(n_neurons**2))
    V_threshold = par['V_threshold'] 
    voltages = np.zeros(n_neurons*n_timesteps).reshape(n_neurons, n_timesteps)
    t = 0
    av_V = 0
    while ((t < n_timesteps - 1 - max_del) and (av_V < V_threshold)):
       # print(3)
        states, input_matrix, voltages = one_step_LIF_noise(states, input_matrix, voltages, syn_del, W_matrix,  edges, t, par, noise_freq, noise_vector)
      #  print(4)
        t = t + 1
        sum_V = 0
        for i in range(n_neurons):
            sum_V = sum_V + voltages[i,t]
        av_V = sum_V / n_neurons    
        #print(av_V)
    return(states, voltages, t)    




In [32]:
#to get pattern only till the burst, rename full_process_no_stop  to full_process
states, voltages, t = full_process( edges, syn_del, input_matrix, W_matrix,  par, n_timesteps, noise_freq, noise_vector)
#df_voltages = pd.DataFrame(voltages)
#df_states = pd.DataFrame(states)

#df_states.to_csv("states.csv")
#df_voltages.to_csv("voltages.csv")

#plt.plot(voltages)
#plt.savefig("voltages.svg")

NameError: name 'n_timesteps' is not defined

In [89]:
print(t)

816


In [35]:
number_of_slices = 5
for i in range(number_of_slices):  
    
    a = str(i)
    filename_prob = "uniform_times_and_probs-" + a + ".csv"
    filename_weights = "uniform_weights-" + a + ".csv"
    filename_activated = "uniform_activated-" + a + ".csv"
    
    M = getRandomConnectivity(N,p) #prepare connectivity matrix of the network. In this case ER matrix.
    edges = matrixOfEdges(M,N)  #other form of this matrix
    W_matrix = weight_matrix(n_neurons, edges, mu,sigma) #matrix of synaptic weights


    
    syn_del = prepare_syndel_2(avdel, stddel, stepsize, edges, n_neurons ) #matrix of synaptic delays 
    
    df_weights = pd.DataFrame(W_matrix )

    df_weights.to_csv(filename_weights)
    
    df_activated = pd.DataFrame(set_activated)
    df_activated.to_csv(filename_activated)


    n_trials = 20
    probs, times = one_slice_experiment_2(edges, syn_del, W_matrix,  parameters, V, period, stdlaser, set_activated, n_trials, noise_freq, noise_vector)
#n_activated = 20
#from google.colab import files
    print(probs)
    print(times)
    l = len(probs)
    dots = []
    

                                
    for j in range(l):
      dots.append(n_activated - j)
    
                                      
    my_dict = {}
    my_dict["Probabilities"] = probs
    my_dict["Times"] = times
    my_dict["Number of neurons"] = dots

    df = pd.DataFrame(my_dict)

    df.to_csv(filename_prob)

    #files.download('probs_and_times.csv')





[1.0, 1.0, 0.85, 0.35, 0.05]
[1547.65, 1821.2, 2202.3529411764707, 2760.285714285714, 3339.0]
[1.0, 1.0, 0.9, 0.7, 0.05]
[1610.6, 1634.9, 1606.3333333333333, 2458.785714285714, 1613.0]
[1.0, 1.0, 1.0, 0.8, 0.35, 0.1]
[1010.2, 1423.2, 1561.05, 2314.75, 3254.285714285714, 3089.0]
[1.0, 1.0, 0.9, 0.65, 0.45, 0.3, 0.2, 0]
[1322.55, 1581.45, 2445.8333333333335, 2280.923076923077, 2059.5555555555557, 2095.6666666666665, 3135.25, 0]
[1.0, 1.0, 0.9, 0.4, 0.05]
[1298.05, 1580.7, 2134.6666666666665, 1673.375, 1576.0]
